In [10]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gensim.models.keyedvectors as kv
import pickle
from tensorflow import keras
from tensorflow.keras import layers

In [40]:
df_train = pd.read_csv(r'C:\Users\prita\Desktop\fake-news\train.csv')

In [41]:
df_train

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [42]:
df_train.drop('id', axis=1, inplace=True)

In [43]:
df_train

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [44]:
df_train['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [45]:
#df_train = df_train[0:10000].copy()

In [46]:
df_train

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [47]:
df_train['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [48]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20242 non-null  object
 1   author  18843 non-null  object
 2   text    20761 non-null  object
 3   label   20800 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 650.1+ KB


In [49]:
df_train.isna().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [50]:
df_train

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [24]:
df_train['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [52]:
sentence = nltk.sent_tokenize(df_train['text'][0])

In [53]:
len(sentence)

37

In [44]:
word = [nltk.word_tokenize(sentence) for sentence in nltk.sent_tokenize(df_train['text'][0])]

In [45]:
word

[['House',
  'Dem',
  'Aide',
  ':',
  'We',
  'Didn',
  '’',
  't',
  'Even',
  'See',
  'Comey',
  '’',
  's',
  'Letter',
  'Until',
  'Jason',
  'Chaffetz',
  'Tweeted',
  'It',
  'By',
  'Darrell',
  'Lucus',
  'on',
  'October',
  '30',
  ',',
  '2016',
  'Subscribe',
  'Jason',
  'Chaffetz',
  'on',
  'the',
  'stump',
  'in',
  'American',
  'Fork',
  ',',
  'Utah',
  '(',
  'image',
  'courtesy',
  'Michael',
  'Jolley',
  ',',
  'available',
  'under',
  'a',
  'Creative',
  'Commons-BY',
  'license',
  ')',
  'With',
  'apologies',
  'to',
  'Keith',
  'Olbermann',
  ',',
  'there',
  'is',
  'no',
  'doubt',
  'who',
  'the',
  'Worst',
  'Person',
  'in',
  'The',
  'World',
  'is',
  'this',
  'week–FBI',
  'Director',
  'James',
  'Comey',
  '.'],
 ['But',
  'according',
  'to',
  'a',
  'House',
  'Democratic',
  'aide',
  ',',
  'it',
  'looks',
  'like',
  'we',
  'also',
  'know',
  'who',
  'the',
  'second-worst',
  'person',
  'is',
  'as',
  'well',
  '.'],
 ['It

In [54]:
w2v = Word2Vec(sentence, min_count=1)

In [55]:
w2v.wv.vectors

array([[-0.15271856,  0.2502758 , -0.01540853, ..., -0.02047805,
        -0.01122011,  0.05150572],
       [-0.15566128,  0.24419405, -0.01498001, ..., -0.01515269,
        -0.02226613,  0.04918297],
       [-0.10776014,  0.18010774, -0.02155565, ..., -0.00988711,
        -0.00097592,  0.02497094],
       ...,
       [-0.01442759,  0.03302928, -0.00299068, ..., -0.00638397,
         0.00454429, -0.00152761],
       [-0.02342624,  0.03288148, -0.00553077, ..., -0.01023269,
         0.00512497, -0.00151036],
       [-0.00366607,  0.00641041, -0.00150501, ..., -0.00349541,
        -0.00483839, -0.00335302]], dtype=float32)

In [56]:
len(w2v.wv.vectors)

74

In [51]:
#df_train['full'] = df_train['author'] + '.' + ' ' + '\n' + df_train['title'] + '.' + ' ' + '\n' + df_train['text']
df_train['full'] = df_train['author'] + '.' + ' ' + '\n' + df_train['title']

In [52]:
df_train

,title,author,text,label,full
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus. \nHouse Dem Aide: We Didn’t Eve...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn. \nFLYNN: Hillary Clinton, Big..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com. \nWhy the Truth Might Get ...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss. \n15 Civilians Killed In Sing...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy. \nIranian woman jailed for fic...
...,...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson. \nRapper T.I.: Trump a ’Poster ...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman. \nN.F.L. Playoffs: Schedule,..."
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams. \nM...
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary. \nNATO, Russia To Hold Parallel E..."


In [53]:
df_train.drop(['title', 'author', 'text'], axis=1, inplace=True)
#df_train.drop(['author', 'text'], axis=1, inplace=True)

In [54]:
df_train.head(10)

,label,full
0,1,Darrell Lucus. \nHouse Dem Aide: We Didn’t Eve...
1,0,"Daniel J. Flynn. \nFLYNN: Hillary Clinton, Big..."
2,1,Consortiumnews.com. \nWhy the Truth Might Get ...
3,1,Jessica Purkiss. \n15 Civilians Killed In Sing...
4,1,Howard Portnoy. \nIranian woman jailed for fic...
5,0,Daniel Nussbaum. \nJackie Mason: Hollywood Wou...
6,1,NaN
7,0,Alissa J. Rubin. \nBenoît Hamon Wins French So...
8,0,NaN
9,0,Megan Twohey and Scott Shane. \nA Back-Channel...


In [110]:
nltk.sent_tokenize(df_train['full'][0])

['Darrell Lucus.',
 'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It.',
 'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey.',
 'But according to a House Democratic aide, it looks like we also know who the second-worst person is as well.',
 'It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey.',
 'They found out via a tweet from one of the Republican committee chairmen.',
 'As we now know, Comey notified the Republican chairmen and Dem

In [70]:
df_train.isna().sum()

title    0
label    0
dtype: int64

In [112]:
np.where(pd.isnull(df_train['full']))

(array([    6,     8,    20, ..., 20771, 20772, 20786], dtype=int64),)

In [55]:
df_train.dropna(inplace=True)

In [56]:
df_train.reset_index(inplace=True)

In [57]:
df_train.drop('index', axis=1, inplace=True)

In [58]:
df_train

,label,full
0,1,Darrell Lucus. \nHouse Dem Aide: We Didn’t Eve...
1,0,"Daniel J. Flynn. \nFLYNN: Hillary Clinton, Big..."
2,1,Consortiumnews.com. \nWhy the Truth Might Get ...
3,1,Jessica Purkiss. \n15 Civilians Killed In Sing...
4,1,Howard Portnoy. \nIranian woman jailed for fic...
...,...,...
18280,0,Jerome Hudson. \nRapper T.I.: Trump a ’Poster ...
18281,0,"Benjamin Hoffman. \nN.F.L. Playoffs: Schedule,..."
18282,0,Michael J. de la Merced and Rachel Abrams. \nM...
18283,1,"Alex Ansary. \nNATO, Russia To Hold Parallel E..."


In [59]:
labels = df_train.pop('label')

In [60]:
labels

0        1
1        0
2        1
3        1
4        1
        ..
18280    0
18281    0
18282    0
18283    1
18284    1
Name: label, Length: 18285, dtype: int64

In [75]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prita\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [61]:
lemma = WordNetLemmatizer()
stem = PorterStemmer()
snowstem = SnowballStemmer('english')

In [33]:
test = [] # list of list of lists
for row in range(0, 2):
    sentences = nltk.sent_tokenize(df_train['full'][row])
    sent = []
    for i in range(0, len(sentences)):
        sentences[i] = re.sub('[^a-zA-Z0-9]', ' ', sentences[i])
        sentences[i] = sentences[i].lower()
        words = nltk.word_tokenize(sentences[i])
        sentences[i] = [snowstem.stem(word) for word in words if word not in stopwords.words('english')]
        #sentences[i] = ' '.join(sentences[i])
    test.append(sentences)

In [151]:
test = [] # list of lists
for row in range(0, 2):
    sentences = nltk.sent_tokenize(df_train['full'][row])
    sent = []
    for i in range(0, len(sentences)):
        sentences[i] = re.sub('[^a-zA-Z0-9]', ' ', sentences[i])
        sentences[i] = sentences[i].lower()
        words = nltk.word_tokenize(sentences[i])
        for word in words: 
            if word not in stopwords.words('english'):
                sent.append(snowstem.stem(word))
    test.append(sent)

In [152]:
len(test)

2

In [32]:
test

[['darrel',
  'lucus',
  'hous',
  'dem',
  'aid',
  'even',
  'see',
  'comey',
  'letter',
  'jason',
  'chaffetz',
  'tweet',
  'hous',
  'dem',
  'aid',
  'even',
  'see',
  'comey',
  'letter',
  'jason',
  'chaffetz',
  'tweet',
  'darrel',
  'lucus',
  'octob',
  '30',
  '2016',
  'subscrib',
  'jason',
  'chaffetz',
  'stump',
  'american',
  'fork',
  'utah',
  'imag',
  'courtesi',
  'michael',
  'jolley',
  'avail',
  'creativ',
  'common',
  'licens',
  'apolog',
  'keith',
  'olbermann',
  'doubt',
  'worst',
  'person',
  'world',
  'week',
  'fbi',
  'director',
  'jame',
  'comey',
  'accord',
  'hous',
  'democrat',
  'aid',
  'look',
  'like',
  'also',
  'know',
  'second',
  'worst',
  'person',
  'well',
  'turn',
  'comey',
  'sent',
  'infam',
  'letter',
  'announc',
  'fbi',
  'look',
  'email',
  'may',
  'relat',
  'hillari',
  'clinton',
  'email',
  'server',
  'rank',
  'democrat',
  'relev',
  'committe',
  'hear',
  'comey',
  'found',
  'via',
  'tweet'

In [52]:
df_train['full'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [53]:
sentences

['hous dem aid even see comey letter jason chaffetz tweet']

In [23]:
#from sklearn.model_selection import train_test_split

In [24]:
#x_train, y_train, x_test, y_test = train_test_split(df_train, labels, test_size = 0.3, shuffle=True, random_state=42)

In [25]:
#x_train

,full
4194,Breitbart News. \nBill Cosby’s Freedom and Leg...
259,Emma G. Fitzsimmons. \nM.T.A. Shortens L Train...
2163,The New York Times. \nA Night of Tornadoes: 3 ...
7887,"Rachel Abrams. \nDespite a Trust, Ivanka Trump..."
380,"Mark Bittman. \nWhen Cooking, Invest Time. Or ..."
...,...
5734,Andrew Meyer. \nThis Is Why America Will Get t...
5191,"James Poniewozik. \nThe 2017 Class Divide, as ..."
5390,Alison Smale. \nSailing Couple Seized by Somal...
860,Michelle Moons. \nPresident Trump Honors Littl...


In [37]:
text = []
for row in range(0, len(x_train)):
    sentences = nltk.sent_tokenize(df_train['full'][row])
    for i in range(0, len(sentences)):
        sentences[i] = re.sub('[^a-zA-Z0-9]', ' ', sentences[i])
        sentences[i] = sentences[i].lower()
        words = nltk.word_tokenize(sentences[i])
        sentences[i] = [snowstem.stem(word) for word in words if word not in stopwords.words('english')]
        #sentences[i] = ' '.join(sentences[i])
        text.append(sentences[i])

In [31]:
text = [] # list of lists
for row in range(0, len(df_train)):
    sentences = nltk.sent_tokenize(df_train['full'][row])
    sent = []
    for i in range(0, len(sentences)):
        sentences[i] = re.sub('[^a-zA-Z0-9]', ' ', sentences[i])
        sentences[i] = sentences[i].lower()
        words = nltk.word_tokenize(sentences[i])
        for word in words: 
            if word not in stopwords.words('english'):
                sent.append(snowstem.stem(word))
    text.append(sent)

In [32]:
len(text)

18285

In [33]:
# with open('w2vtextlistoflists.pkl', 'wb') as f:
#     pickle.dump(text, f)

In [62]:
with open('w2vtextlistoflists.pkl', 'rb') as f:
    text = pickle.load(f)

In [63]:
len(text)

18285

In [36]:
w2v = Word2Vec(text, min_count=1, vector_size=1000, sg=0)

In [37]:
x = w2v.wv.vectors

In [38]:
x

array([[ 6.60127640e-01, -1.12145342e-01,  1.36173218e-01, ...,
        -1.06131256e-01, -3.15022692e-02, -7.12141320e-02],
       [ 7.36618519e-01, -1.29526749e-01,  2.90710926e-01, ...,
        -6.20273836e-02,  4.92969118e-02,  4.01737727e-02],
       [ 6.50571167e-01, -1.48232549e-01,  7.20724761e-02, ...,
        -3.12338412e-01, -2.82015085e-01, -3.38000096e-02],
       ...,
       [ 5.77555643e-03, -3.23338812e-04, -2.11532414e-03, ...,
        -1.92119053e-03, -3.36242188e-03, -3.83379054e-03],
       [ 6.08366355e-03, -1.30249327e-03,  3.56789166e-03, ...,
        -2.52600992e-04, -4.63850156e-04,  1.41466735e-03],
       [ 3.12533439e-03, -1.03773875e-03, -5.68405492e-04, ...,
        -4.43479774e-04, -9.89438850e-04, -5.99803578e-04]], dtype=float32)

In [41]:
len(x)

16397

In [42]:
w2v.wv.most_similar('girl')

[('last', 0.9999181628227234),
 ('studi', 0.9998908638954163),
 ('store', 0.9998868107795715),
 ('cancer', 0.9998592734336853),
 ('wear', 0.9998363852500916),
 ('sentenc', 0.9998340606689453),
 ('market', 0.9998233914375305),
 ('suspect', 0.9998214244842529),
 ('sever', 0.9998153448104858),
 ('fli', 0.9998108744621277)]

In [43]:
#w2v.wv.save_word2vec_format('w2vlistoflists.bin')

In [88]:
text_vector = kv.KeyedVectors.load_word2vec_format('w2vlistoflists.bin')

In [160]:
text_vector

In [91]:
len(text_vector.key_to_index)

74995

In [29]:
w2v.wv['girl']

NameError: name 'w2v' is not defined

In [42]:
w2v.wv.index_to_key

['new',
 'time',
 'york',
 'trump',
 'breitbart',
 'hillari',
 'clinton',
 'donald',
 'elect',
 'u',
 'say',
 'michael',
 'obama',
 'news',
 'john',
 'com',
 'presid',
 'state',
 'video',
 'war',
 'world',
 'report',
 'russia',
 'vote',
 'us',
 'attack',
 'comment',
 'year',
 'admin',
 'american',
 'day',
 'call',
 'hous',
 'blogger',
 'email',
 'norepli',
 'fbi',
 'get',
 'media',
 'america',
 'white',
 'david',
 'kill',
 'polic',
 'key',
 'daniel',
 'bill',
 'charli',
 'campaign',
 'plan',
 'democrat',
 'pam',
 'protest',
 'e',
 'support',
 'first',
 'one',
 'make',
 'brief',
 'show',
 'robert',
 'mike',
 'jame',
 'black',
 'even',
 'win',
 'anti',
 '2016',
 'syria',
 'right',
 'j',
 'back',
 'man',
 'take',
 'watch',
 'nation',
 'like',
 'peopl',
 'russian',
 'paul',
 'go',
 'b',
 '2',
 'die',
 'investig',
 'break',
 'use',
 'post',
 'may',
 'n',
 'de',
 'ben',
 '3',
 'ian',
 'muslim',
 'million',
 'polit',
 'immigr',
 'could',
 'andrew',
 'want',
 'eric',
 'author',
 'chang',
 'c',

In [52]:
from sklearn.decomposition import PCA
from matplotlib import pyplot

In [42]:
X = w2v.wv.key_to_index

In [44]:
w2v.wv.key_to_index

{'new': 0,
 'time': 1,
 'york': 2,
 'trump': 3,
 'breitbart': 4,
 'hillari': 5,
 'clinton': 6,
 'donald': 7,
 'elect': 8,
 'u': 9,
 'say': 10,
 'michael': 11,
 'obama': 12,
 'news': 13,
 'john': 14,
 'com': 15,
 'presid': 16,
 'state': 17,
 'video': 18,
 'war': 19,
 'world': 20,
 'report': 21,
 'russia': 22,
 'vote': 23,
 'us': 24,
 'attack': 25,
 'comment': 26,
 'year': 27,
 'admin': 28,
 'american': 29,
 'day': 30,
 'call': 31,
 'hous': 32,
 'blogger': 33,
 'email': 34,
 'norepli': 35,
 'fbi': 36,
 'get': 37,
 'media': 38,
 'america': 39,
 'white': 40,
 'david': 41,
 'kill': 42,
 'polic': 43,
 'key': 44,
 'daniel': 45,
 'bill': 46,
 'charli': 47,
 'campaign': 48,
 'plan': 49,
 'democrat': 50,
 'pam': 51,
 'protest': 52,
 'e': 53,
 'support': 54,
 'first': 55,
 'one': 56,
 'make': 57,
 'brief': 58,
 'show': 59,
 'robert': 60,
 'mike': 61,
 'jame': 62,
 'black': 63,
 'even': 64,
 'win': 65,
 'anti': 66,
 '2016': 67,
 'syria': 68,
 'right': 69,
 'j': 70,
 'back': 71,
 'man': 72,
 'take'

In [88]:
tokenizer = Tokenizer(oov_token=True)
tokenizer.fit_on_texts(text)

text_tokenizer = tokenizer.texts_to_sequences(text)

In [89]:
train_token

In [90]:
len(text_tokenizer)

18285

In [91]:
max([len(x) for x in text_tokenizer])

51

In [92]:
# text_tokenizer_pad = pad_sequences(text_tokenizer, maxlen=max([len(x) for x in text_tokenizer]))
text_tokenizer_pad = pad_sequences(text_tokenizer, maxlen=100)

In [93]:
(text_tokenizer_pad)

array([[   0,    0,    0, ...,  138, 3928,  597],
       [   0,    0,    0, ...,  230, 1083,    6],
       [   0,    0,    0, ..., 1180,   39,  121],
       ...,
       [   0,    0,    0, ...,    2,    4,    3],
       [   0,    0,    0, ..., 5344, 3244, 6352],
       [   0,    0,    0, ...,  120, 2620, 1944]])

In [94]:
text_tokenizer_pad.shape

(18285, 100)

In [95]:
tokenizer.word_index

{True: 1,
 'new': 2,
 'time': 3,
 'york': 4,
 'trump': 5,
 'breitbart': 6,
 'hillari': 7,
 'clinton': 8,
 'donald': 9,
 'elect': 10,
 'u': 11,
 'say': 12,
 'michael': 13,
 'obama': 14,
 'news': 15,
 'john': 16,
 'com': 17,
 'presid': 18,
 'state': 19,
 'video': 20,
 'war': 21,
 'world': 22,
 'russia': 23,
 'report': 24,
 'vote': 25,
 'us': 26,
 'attack': 27,
 'comment': 28,
 'year': 29,
 'admin': 30,
 'american': 31,
 'day': 32,
 'call': 33,
 'hous': 34,
 'blogger': 35,
 'email': 36,
 'norepli': 37,
 'fbi': 38,
 'get': 39,
 'media': 40,
 'white': 41,
 'america': 42,
 'david': 43,
 'kill': 44,
 'polic': 45,
 'key': 46,
 'daniel': 47,
 'bill': 48,
 'campaign': 49,
 'charli': 50,
 'plan': 51,
 'democrat': 52,
 'pam': 53,
 'protest': 54,
 'e': 55,
 'support': 56,
 'first': 57,
 'one': 58,
 'make': 59,
 'brief': 60,
 'show': 61,
 'robert': 62,
 'mike': 63,
 'jame': 64,
 'black': 65,
 'even': 66,
 'win': 67,
 'anti': 68,
 '2016': 69,
 'syria': 70,
 'right': 71,
 'j': 72,
 'back': 73,
 'man':

In [50]:
# Adding 1 because of reserved 0 index
# Embedding Layer creates one more vector for "UNKNOWN" words, or padded words (0s). This Vector is filled with zeros.
# Thus our vocab size inceeases by 1
weight_matrix = np.zeros((len(tokenizer.word_index) + 1, 1000))

How the size of the weight matrix is determined ?

Each word has been represented as index values such as 0,1,2,3,.... So the total words here is 16397 and each each word been represented from 0 to 16396. If I consider a matrix, the size will be 16397x16397. Each word has been represented as a 1000 dimensional vector. The matrix size will be 16397x1000. So the weight matrix size will be
16397x16397 * 16397x1000 = 16397x1000. But since there will be one additional row for the all the unknown words in embedding, the actual size will be (16397+1)x1000. 
And also since we are passing the weight matrix as not learning them from scratch, we are using `trainable=False` in parameters.

Why the `text_tokenizer` and `padding` is used ?

Since text words cannot be given to ML or DL models, we `tokenize` each word with a unique number. Don't confuse unique number with index number. In Word2Vec we give indexing to each word. But in `tokenize` we give unique number to each word more like ordinal encoding from sklearn. But the thing is since each sentence (which will be treated as rows in models) is not of same length, we cannot pass it to the model. For that we used `pad_sequence` to bring all the row of the same length.

In [39]:
weight_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
weight_matrix.shape

(16398, 1000)

In [43]:
for word, i in tokenizer.word_index.items():
    weight_matrix[i] = X[word]
    #print(weight_matrix[i], X[word])

In [44]:
weight_matrix

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.0000e+00, 1.0000e+00, 1.0000e+00, ..., 1.0000e+00, 1.0000e+00,
        1.0000e+00],
       ...,
       [1.0479e+04, 1.0479e+04, 1.0479e+04, ..., 1.0479e+04, 1.0479e+04,
        1.0479e+04],
       [1.4162e+04, 1.4162e+04, 1.4162e+04, ..., 1.4162e+04, 1.4162e+04,
        1.4162e+04],
       [1.6396e+04, 1.6396e+04, 1.6396e+04, ..., 1.6396e+04, 1.6396e+04,
        1.6396e+04]])

In [96]:
from sklearn.model_selection import train_test_split

In [97]:
x_train, x_test, y_train, y_test = train_test_split(text_tokenizer_pad, labels, test_size = 0.2, shuffle=True, random_state=42)

In [98]:
len(x_train)

14628

In [99]:
len(y_train)

14628

In [100]:
# input = keras.Input(shape=(51,))
# x = layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=1000, weights=[weight_matrix], input_length=max([len(x) for x in text_tokenizer]), trainable=False)(input)
# x = layers.Conv1D(filters=5, kernel_size=3, activation='relu')(x)
# x = layers.MaxPool1D(pool_size=3, strides=1, padding='same')(x)
# x = layers.LSTM(512, return_sequences=True)(x)
# output = layers.Dense(1, activation='sigmoid')(x)

input = keras.Input(shape=(100,))
x = layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=1000, input_length=max([len(x) for x in text_tokenizer]), trainable=True)(input)
x = layers.Conv1D(filters=5, kernel_size=3, activation='relu')(x)
x = layers.MaxPool1D(pool_size=3, strides=1, padding='same')(x)
x = layers.LSTM(512, return_sequences=False)(x)
output = layers.Dense(1, activation='sigmoid')(x)

In [101]:
model = keras.Model(inputs=input, outputs=output, name='Fake_News_Classifier')

In [102]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [103]:
hist = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), shuffle=True)

Epoch 1/5
458/458 [==============================] - 958s 2s/step - loss: 0.1127 - accuracy: 0.9470 - val_loss: 0.0378 - val_accuracy: 0.9896
Epoch 2/5
458/458 [==============================] - 867s 2s/step - loss: 0.0148 - accuracy: 0.9966 - val_loss: 0.0201 - val_accuracy: 0.9951
Epoch 3/5
458/458 [==============================] - 762s 2s/step - loss: 0.0059 - accuracy: 0.9989 - val_loss: 0.0169 - val_accuracy: 0.9967
Epoch 4/5
458/458 [==============================] - 765s 2s/step - loss: 0.0221 - accuracy: 0.9919 - val_loss: 0.0180 - val_accuracy: 0.9951
Epoch 5/5
458/458 [==============================] - 762s 2s/step - loss: 0.0035 - accuracy: 0.9992 - val_loss: 0.0233 - val_accuracy: 0.9948


In [104]:
#model.save('FakeNewsClassifier.h5')

In [106]:
with open('tokenize_file.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [59]:
score = model.predict(x_test)

In [60]:
len(score[0])

1

In [61]:
from sklearn.metrics import confusion_matrix

In [62]:
for i, item in enumerate(score):
    if item >= 0.5:
        score[i] = 1
    else:
        score[i] = 0

In [63]:
confusion_matrix(y_test, score, labels=[0,1])

array([[3101,    6],
       [  13, 2366]], dtype=int64)

# Test Data

In [107]:
df_test = pd.read_csv(r'C:\Users\prita\Desktop\fake-news\test.csv')

In [108]:
df_test

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [109]:
#df_train['full'] = df_train['author'] + '.' + ' ' + '\n' + df_train['title'] + '.' + ' ' + '\n' + df_train['text']
df_test['full'] = df_test['author'] + '.' + ' ' + '\n' + df_test['title']

In [110]:
df_test

,id,title,author,text,full
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",David Streitfeld. \nSpecter of Trump Loosens T...
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,NaN
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,Common Dreams. \n#NoDAPL: Native American Lead...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",Daniel Victor. \nTim Tebow Will Attempt Anothe...
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Truth Broadcast Network. \nKeiser Report: Meme...
...,...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...,Jody Rosen. \nThe Bangladeshi Traffic Jam That...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...,Sheryl Gay Stolberg. \nJohn Kasich Signs One A...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...,"Mike McPhate. \nCalifornia Today: What, Exactl..."
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...,NaN


In [111]:
df_test.drop(['title', 'author', 'text'], axis=1, inplace=True)
#df_train.drop(['author', 'text'], axis=1, inplace=True)

In [112]:
df_test.head(10)

,id,full
0,20800,David Streitfeld. \nSpecter of Trump Loosens T...
1,20801,NaN
2,20802,Common Dreams. \n#NoDAPL: Native American Lead...
3,20803,Daniel Victor. \nTim Tebow Will Attempt Anothe...
4,20804,Truth Broadcast Network. \nKeiser Report: Meme...
5,20805,NaN
6,20806,Pam Key. \nPelosi Calls for FBI Investigation ...
7,20807,Trevor Loudon. \nWeekly Featured Profile – Ran...
8,20808,NaN
9,20809,NaN


In [113]:
df_test.drop('id', axis=1, inplace=True)

In [114]:
df_test.dropna(inplace=True)

In [115]:
df_test.reset_index(inplace=True)

In [116]:
df_test.drop('index', axis=1, inplace=True)

In [117]:
df_test

,full
0,David Streitfeld. \nSpecter of Trump Loosens T...
1,Common Dreams. \n#NoDAPL: Native American Lead...
2,Daniel Victor. \nTim Tebow Will Attempt Anothe...
3,Truth Broadcast Network. \nKeiser Report: Meme...
4,Pam Key. \nPelosi Calls for FBI Investigation ...
...,...
4570,Pam Key. \nTrump on If ’Tapes’ Exist of Comey ...
4571,Jody Rosen. \nThe Bangladeshi Traffic Jam That...
4572,Sheryl Gay Stolberg. \nJohn Kasich Signs One A...
4573,"Mike McPhate. \nCalifornia Today: What, Exactl..."


In [118]:
text_test = [] # list of lists
for row in range(0, len(df_test)):
    sentences = nltk.sent_tokenize(df_test['full'][row])
    sent = []
    for i in range(0, len(sentences)):
        sentences[i] = re.sub('[^a-zA-Z0-9]', ' ', sentences[i])
        sentences[i] = sentences[i].lower()
        words = nltk.word_tokenize(sentences[i])
        for word in words: 
            if word not in stopwords.words('english'):
                sent.append(snowstem.stem(word))
    text_test.append(sent)

In [119]:
text_test[0]

['david',
 'streitfeld',
 'specter',
 'trump',
 'loosen',
 'tongu',
 'purs',
 'string',
 'silicon',
 'valley',
 'new',
 'york',
 'time']

In [120]:
len(text_test)

4575

In [121]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(text)

text_test_tokenizer = tokenizer.texts_to_sequences(text_test)

In [122]:
text_test_tokenizer[0]

[43, 3363, 1, 5, 5090, 8238, 9207, 9320, 2431, 2026, 2, 4, 3]

In [123]:
len(text_test_tokenizer)

4575

In [124]:
max([len(x) for x in text_test_tokenizer])

48

In [125]:
max([len(x) for x in text_tokenizer])

51

In [129]:
text_test_tokenizer_pad = pad_sequences(text_test_tokenizer, maxlen=100)

In [130]:
(text_test_tokenizer_pad)

array([[   0,    0,    0, ...,    2,    4,    3],
       [   0,    0,    0, ..., 1080, 1176,   45],
       [   0,    0,    0, ...,    2,    4,    3],
       ...,
       [   0,    0,    0, ...,    2,    4,    3],
       [   0,    0,    0, ...,    2,    4,    3],
       [   0,    0,    0, ...,    2,    4,    3]])

In [131]:
score_test = model.predict(text_test_tokenizer_pad)

In [132]:
score_test

array([[6.2333536e-05],
       [9.9990141e-01],
       [1.4150920e-06],
       ...,
       [9.0808171e-06],
       [1.8710342e-06],
       [3.7361599e-05]], dtype=float32)

In [134]:
score_test[0][0]

6.2333536e-05

In [141]:
# for i, item in enumerate(score_test):
#     if item >= 0.5:
#         score_test[i] = 1
#     else:
#         score_test[i] = 0

for i, item in enumerate(score_test):
    #print(item[0])
    print(f'0-Real: {int((1-item[0]) * 100}, 1-Fake: {item[0] * 100}')

0: 99.99376664636657, 1: 0.00623335363343358
0: 0.00985860824584961, 1: 99.99014139175415
0: 99.99985849079849, 1: 0.00014150920151223545
0: 0.015312433242797852, 1: 99.9846875667572
0: 99.9976671380864, 1: 0.0023328619136009365
0: 0.024831295013427734, 1: 99.97516870498657
0: 0.008475780487060547, 1: 99.99152421951294
0: 0.06178617477416992, 1: 99.93821382522583
0: 0.008869171142578125, 1: 99.99113082885742
0: 0.008493661880493164, 1: 99.9915063381195
0: 0.008392333984375, 1: 99.99160766601562
0: 99.99959686706461, 1: 0.0004031329353892943
0: 99.999510683665, 1: 0.0004893163350061513
0: 99.99752969870315, 1: 0.0024703012968529947
0: 0.008571147918701172, 1: 99.9914288520813
0: 99.99556221846433, 1: 0.004437781535671093
0: 0.008571147918701172, 1: 99.9914288520813
0: 0.033152103424072266, 1: 99.96684789657593
0: 0.018656253814697266, 1: 99.9813437461853
0: 0.01386404037475586, 1: 99.98613595962524
0: 99.99688514326408, 1: 0.003114856735919602
0: 0.008153915405273438, 1: 99.991846084594

In [91]:
score_test

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [1]:
import random

In [7]:
dfdummy = [random.choice([0, 1]) for _ in range(4575)]

In [8]:
dfdummy

[0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,


In [9]:
len(dfdummy)

4575

In [11]:
df_dummy = pd.DataFrame(columns=['labels'])

In [12]:
df_dummy['labels'] = dfdummy

In [13]:
df_dummy

,labels
0,0
1,0
2,1
3,1
4,1
...,...
4570,0
4571,1
4572,1
4573,1


In [14]:
values, counts = np.unique(dfdummy, return_counts=True)

In [15]:
values, counts

(array([0, 1]), array([2253, 2322], dtype=int64))

In [38]:
df_dummy[0:30].to_csv('test_labels.csv', index=False)

In [39]:
df_test[0:30].to_csv('test.csv', index=False)

In [26]:
df_test.columns.values

array(['id', 'title', 'author', 'text'], dtype=object)

In [29]:
list(df_test.select_dtypes(include=['object']))

['title', 'author', 'text']

In [36]:
obj_col = list(df_test.select_dtypes(include=['object']))
if len(obj_col) > 1:
    hel = ''
    for column in ['title', 'author']:
        hel += df_test[column] + '.' + ' ' + '\n'
hel

0       Specter of Trump Loosens Tongues, if Not Purse...
1       #NoDAPL: Native American Leaders Vow to Stay A...
2       Tim Tebow Will Attempt Another Comeback, This ...
3       Keiser Report: Meme Wars (E995). \nTruth Broad...
4       Pelosi Calls for FBI Investigation to Find Out...
                              ...                        
4570    Trump on If ’Tapes’ Exist of Comey Conversatio...
4571    The Bangladeshi Traffic Jam That Never Ends - ...
4572    John Kasich Signs One Abortion Bill in Ohio bu...
4573    California Today: What, Exactly, Is in Your Su...
4574    Awkward Sex, Onscreen and Off - The New York T...
Name: title, Length: 4575, dtype: object

In [37]:
hel[0]

'Specter of Trump Loosens Tongues, if Not Purse Strings, in Silicon Valley - The New York Times. \nDavid Streitfeld. \n'